In [ ]:
from pyspark.sql import SparkSession
from functools import reduce
import pyspark.sql.functions as F
import pyspark.sql.types as T
import pandas as pd

spark = SparkSession.builder.appName("Ch09 - ex09").getOrCreate()

In [ ]:
# ex 9.1
ex9_1 = pd.Series(["red", "blue", "blue", "yellow"])

def color_to_num(colors: pd.Series) -> pd.Series:
    return colors.apply(
        lambda x: {"red": 1, "blue": 2, "yellow": 3}.get(x)
    )

In [ ]:
color_to_num(ex9_1)

In [ ]:
color_to_num_udf = F.pandas_udf(color_to_num, T.IntegerType())
ex9_1_df = spark.createDataFrame(ex9_1.to_frame())

In [ ]:
ex9_1_df.show()

In [ ]:
ex9_1_df.select(color_to_num_udf(F.col("0")).alias("num")).show(5)

In [ ]:
spark.stop()